In [1]:
from numerapi import NumerAPI
import pandas as pd
import json
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
from numerapi import NumerAPI
import pandas as pd
import json

DATA_VERSION = "v5.0"
NUM_ERAS = 159

napi = NumerAPI()
napi.download_dataset(f"{DATA_VERSION}/train.parquet")
napi.download_dataset(f"{DATA_VERSION}/features.json");

feature_metadata = json.load(open(f"{DATA_VERSION}/features.json"))
feature_cols = feature_metadata["feature_sets"]["medium"]
target_cols = feature_metadata["targets"]
training_data = pd.read_parquet(
    f"{DATA_VERSION}/train.parquet",
    columns=["era"] + feature_cols + ['target']
)

# training_data = training_data[training_data["era"].isin(training_data["era"].unique()[::4])]
# training_data[["era"] + feature_cols + ["target"]]

training_data['era'] = training_data['era'].astype(str)

unique_eras = sorted(training_data['era'].unique(), reverse=True)

recent_eras = unique_eras[:NUM_ERAS]

training_data = training_data[training_data['era'].isin(recent_eras)]

2024-11-07 13:46:40,748 INFO numerapi.utils: target file already exists
2024-11-07 13:46:40,751 INFO numerapi.utils: download complete
2024-11-07 13:46:41,153 INFO numerapi.utils: target file already exists
2024-11-07 13:46:41,154 INFO numerapi.utils: download complete


In [4]:
training_data

,era,feature_able_deprived_nona,feature_ablest_inflexional_egeria,feature_absorbable_hyperalgesic_mode,feature_accoutered_revolute_vexillology,feature_acetose_crackerjack_needlecraft,feature_acheulian_conserving_output,feature_acronychal_bilobate_stevenage,feature_acrylic_gallic_wine,feature_adminicular_shod_levant,...,feature_wrapround_chrestomathic_timarau,feature_xanthic_transpadane_saleswoman,feature_xanthochroid_petrified_gutenberg,feature_zincy_cirrhotic_josh,feature_zippy_trine_diffraction,feature_zonal_snuffly_chemism,feature_zygotic_middlebrow_caribbean,feature_zymolytic_intertidal_privet,feature_zymotic_windswept_cooky,target
id,,,,,,,,,,,,,,,,,,,,,
n000eecf2df4ec8f,0416,4,1,0,2,3,2,2,4,1,...,1,3,1,3,4,4,4,2,1,0.25
n0023638dc53f38c,0416,3,4,2,2,0,2,3,3,3,...,3,0,4,0,0,4,0,4,0,0.25
n0030a6f314d32a0,0416,0,4,3,2,1,3,1,3,1,...,2,3,1,4,4,3,3,3,0,0.25
n003366c981485c0,0416,3,0,2,1,2,1,3,1,1,...,4,1,0,1,0,0,3,3,4,0.25
n003d1dc712605e9,0416,4,0,4,3,0,1,0,2,4,...,0,0,4,2,1,0,0,4,3,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffcc1dbdf2212e6,0574,4,4,3,0,2,0,0,2,4,...,0,4,3,1,2,1,4,4,2,0.75
nffde3b371d67394,0574,4,4,3,0,2,4,1,2,4,...,0,4,0,2,2,2,1,3,3,0.25
nfff1a1111b35e84,0574,3,0,1,2,3,2,1,0,1,...,2,4,1,3,4,0,4,0,4,0.50


In [5]:
napi.download_dataset(f"{DATA_VERSION}/live.parquet")

# Load live features
live_features = pd.read_parquet(f"{DATA_VERSION}/live.parquet", columns=feature_cols)
live_features

2024-11-07 13:46:49,389 INFO numerapi.utils: target file already exists
2024-11-07 13:46:49,391 INFO numerapi.utils: starting download
v5.0/live.parquet: 15.7MB [00:00, 22.5MB/s]                   


,feature_able_deprived_nona,feature_ablest_inflexional_egeria,feature_absorbable_hyperalgesic_mode,feature_accoutered_revolute_vexillology,feature_acetose_crackerjack_needlecraft,feature_acheulian_conserving_output,feature_acronychal_bilobate_stevenage,feature_acrylic_gallic_wine,feature_adminicular_shod_levant,feature_adorable_infernal_cartesianism,...,feature_working_jain_acromegaly,feature_wrapround_chrestomathic_timarau,feature_xanthic_transpadane_saleswoman,feature_xanthochroid_petrified_gutenberg,feature_zincy_cirrhotic_josh,feature_zippy_trine_diffraction,feature_zonal_snuffly_chemism,feature_zygotic_middlebrow_caribbean,feature_zymolytic_intertidal_privet,feature_zymotic_windswept_cooky
id,,,,,,,,,,,,,,,,,,,,,
n0039880040fb023,4,0,3,0,2,0,1,1,0,2,...,3,1,3,3,1,3,2,4,0,4
n005b7bee7ec0816,3,1,4,3,2,2,0,0,3,1,...,3,1,2,4,4,1,1,2,1,1
n00693f63f9be56b,2,3,2,0,3,2,2,0,2,0,...,0,3,2,2,3,4,2,2,3,2
n00b8288605a5c6d,2,3,0,2,0,0,4,2,1,0,...,2,1,3,2,1,4,2,0,3,0
n00b97850c393025,4,4,2,4,3,2,1,4,3,3,...,2,3,2,4,2,1,0,4,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffceeac42d236e9,0,0,1,0,3,4,3,1,0,1,...,3,4,2,2,2,2,0,2,1,2
nffcf2396cb13040,4,4,0,4,3,1,3,3,4,0,...,3,4,3,1,4,1,0,4,4,2
nffcfec6bec2fa94,1,0,1,3,3,4,3,3,0,1,...,3,4,2,2,1,2,0,2,0,2


In [6]:

era_grouped_data = {}

for era in recent_eras:
    era_data = training_data[training_data['era'] == era]
    
    features = era_data[feature_cols].values 
    targets = era_data['target'].values
    
    era_grouped_data[era] = (features, targets)


for era, (features, targets) in list(era_grouped_data.items())[:2]:
    print(f"Era: {era}")
    print(f"Features shape: {features.shape}")
    print(f"Targets shape: {targets.shape}")
    print() 


Era: 0574
Features shape: (5580, 705)
Targets shape: (5580,)

Era: 0573
Features shape: (5615, 705)
Targets shape: (5615,)



In [7]:
import numpy as np
import pandas as pd

def calculate_correlation_vector(data, features, target_col):
    correlations = []
    for feature in features:
        correlation = data[feature].corr(target_col)
        correlations.append(correlation)
    
    return np.array(correlations)

def select_top_n_features_by_correlation(era_grouped_data, live_data, target_col, feature_names, top_n=100):
    live_features = live_data.columns
    common_features = list((training_data.columns).intersection(set(live_features)))

    common_feature_indices = [feature_names.index(feature) for feature in common_features]

    print(f"Common features: {common_features} found.")
    
    correlation_sums = np.zeros(len(common_features))

    for era, (features_array, target_array) in era_grouped_data.items():
        era_df = pd.DataFrame(features_array, columns=feature_names)
        target_series = pd.Series(target_array.flatten())
        correlation_vector = calculate_correlation_vector(era_df, common_features, target_series)
        correlation_sums += correlation_vector

    correlation_df = pd.DataFrame({
        'feature': common_features,
        'correlation_sum': correlation_sums
    })
    
    top_features = correlation_df.nlargest(top_n, 'correlation_sum')['feature'].tolist()
    
    return top_features


feature_names = list(training_data.columns.drop(["era", "target"]))

top_100_features = select_top_n_features_by_correlation(era_grouped_data, live_features, "target", feature_names, top_n=100)

print(f"Top 100 features: {top_100_features}")

Common features: ['feature_able_deprived_nona', 'feature_ablest_inflexional_egeria', 'feature_absorbable_hyperalgesic_mode', 'feature_accoutered_revolute_vexillology', 'feature_acetose_crackerjack_needlecraft', 'feature_acheulian_conserving_output', 'feature_acronychal_bilobate_stevenage', 'feature_acrylic_gallic_wine', 'feature_adminicular_shod_levant', 'feature_adorable_infernal_cartesianism', 'feature_adorable_unsuitable_cholecystectomy', 'feature_advertent_deferent_kaif', 'feature_aeneolithic_nineteenth_whipper', 'feature_afoul_drainable_cateran', 'feature_agamid_yuletide_physiology', 'feature_aired_temptable_murmansk', 'feature_alarming_forenamed_shearing', 'feature_aliunde_unhaunted_coacervate', 'feature_altern_packaged_presbyterian', 'feature_analgesic_pensionary_exterior', 'feature_analogical_obstructed_martingale', 'feature_anomic_isocyclic_absinth', 'feature_antediluvian_establishmentarian_zebra', 'feature_anthracoid_tamable_consulship', 'feature_antidepressant_rationalistic_

Top 100 features: ['feature_muscly_splintery_stope', 'feature_venerable_kernelly_durham', 'feature_unmovable_declassified_corrival', 'feature_spendable_peak_omission', 'feature_catechetical_paragogical_accouterment', 'feature_upbeat_boneheaded_chequer', 'feature_haemostatic_pulpiest_pembroke', 'feature_drawn_gimcrack_vulcanalia', 'feature_vestigial_tittering_cyan', 'feature_mycological_rabelaisian_puking', 'feature_substandard_practicable_slobber', 'feature_unimproved_terminist_occupancy', 'feature_unpitying_polyatomic_theodicy', 'feature_preachy_uncontaminated_servitude', 'feature_incognoscible_gangliest_megaphone', 'feature_debonnaire_opulent_stayer', 'feature_axial_solutrean_pry', 'feature_unpuckered_orgasmic_blockbuster', 'feature_choreic_sterilized_lagune', 'feature_mullioned_hidden_niece', 'feature_homier_congestive_queening', 'feature_chattier_tight_academic', 'feature_xanthochroid_petrified_gutenberg', 'feature_velate_abolitionary_definitude', 'feature_proto_rachidial_heldenten

In [8]:
training_data = training_data[["era", "target"] + top_100_features]

In [9]:
training_data

,era,target,feature_muscly_splintery_stope,feature_venerable_kernelly_durham,feature_unmovable_declassified_corrival,feature_spendable_peak_omission,feature_catechetical_paragogical_accouterment,feature_upbeat_boneheaded_chequer,feature_haemostatic_pulpiest_pembroke,feature_drawn_gimcrack_vulcanalia,...,feature_subtriangular_bibliopolic_shuffle,feature_axile_complexioned_tartarization,feature_shouldered_cliffier_chouse,feature_able_deprived_nona,feature_hottish_gutsier_locale,feature_toponymic_bosom_tyre,feature_informed_eminent_chaplain,feature_spluttering_unconvertible_callosity,feature_unfurred_beastlier_spoof,feature_prophylactic_gaulish_nutria
id,,,,,,,,,,,,,,,,,,,,,
n000eecf2df4ec8f,0416,0.25,3,3,0,1,3,1,3,1,...,2,2,2,4,3,3,3,0,2,3
n0023638dc53f38c,0416,0.25,2,2,1,4,1,1,1,3,...,3,4,2,3,4,0,4,0,2,0
n0030a6f314d32a0,0416,0.25,1,0,4,0,3,4,3,1,...,2,2,1,0,4,3,3,1,3,0
n003366c981485c0,0416,0.25,2,2,2,3,3,2,3,2,...,1,0,2,3,1,3,1,3,1,1
n003d1dc712605e9,0416,0.25,1,1,4,4,4,4,4,4,...,3,2,0,4,1,2,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffcc1dbdf2212e6,0574,0.75,1,2,4,2,4,4,4,4,...,0,0,1,4,2,3,0,1,0,1
nffde3b371d67394,0574,0.25,2,1,4,2,3,4,3,4,...,0,0,3,4,2,2,0,2,0,2
nfff1a1111b35e84,0574,0.50,1,1,4,1,4,4,4,2,...,2,0,1,3,2,1,4,1,2,3


In [10]:
class_labels = training_data['target']

class_counts = class_labels.value_counts() 

total_samples = len(class_labels)

class_weights = {cls: total_samples / count for cls, count in class_counts.items()}

weights_tensor = torch.tensor([class_weights[cls] for cls in sorted(class_weights.keys())], dtype=torch.float32).to(device)

print("Class Weights:", class_weights)

Class Weights: {0.5: 2.0000048758249287, 0.25: 4.984663993194799, 0.75: 4.986906252659478, 1.0: 20.213275513723943, 0.0: 20.247698496927217}


In [11]:
from sklearn.preprocessing import StandardScaler

X = training_data[top_100_features].values
y = training_data["target"].values


# scaler = StandardScaler()
# X = scaler.fit_transform(X)

bins = [0, 0.25, 0.5, 0.75, 1]
y = np.digitize(y, bins) - 1 

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [12]:

from pytorch_tabnet.tab_model import TabNetClassifier

tabnet_model = TabNetClassifier(optimizer_params=dict(lr=1e-5))

tabnet_model.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_val, y_val)],
    eval_metric=['accuracy'],
    max_epochs=10,  
    patience=5,     
    batch_size=64,
    virtual_batch_size=128,  
    drop_last=False 
)

epoch 0  | loss: 1.79445 | val_0_accuracy: 0.37021 |  0:02:26s
epoch 1  | loss: 1.53444 | val_0_accuracy: 0.45218 |  0:04:14s
epoch 2  | loss: 1.44068 | val_0_accuracy: 0.47309 |  0:06:01s
epoch 3  | loss: 1.39833 | val_0_accuracy: 0.48562 |  0:07:46s
epoch 4  | loss: 1.37355 | val_0_accuracy: 0.49278 |  0:09:36s
epoch 5  | loss: 1.35708 | val_0_accuracy: 0.49598 |  0:11:26s
epoch 6  | loss: 1.34429 | val_0_accuracy: 0.49826 |  0:13:16s
epoch 7  | loss: 1.33453 | val_0_accuracy: 0.4989  |  0:15:04s
epoch 8  | loss: 1.32631 | val_0_accuracy: 0.49935 |  0:16:53s
epoch 9  | loss: 1.32095 | val_0_accuracy: 0.49954 |  0:18:45s
epoch 10 | loss: 1.31595 | val_0_accuracy: 0.4997  |  0:20:35s
epoch 11 | loss: 1.31271 | val_0_accuracy: 0.49982 |  0:22:25s
epoch 12 | loss: 1.3097  | val_0_accuracy: 0.49986 |  0:24:11s
epoch 13 | loss: 1.30701 | val_0_accuracy: 0.4999  |  0:26:01s
epoch 14 | loss: 1.30492 | val_0_accuracy: 0.49991 |  0:27:53s
epoch 15 | loss: 1.30304 | val_0_accuracy: 0.49997 |  0

KeyboardInterrupt: 

In [13]:
from sklearn.metrics import accuracy_score

preds_val = tabnet_model.predict(X_val)
accuracy = accuracy_score(y_val, preds_val)
print(f"Validation Accuracy: {accuracy}")

Validation Accuracy: 0.5


In [14]:
from sklearn.metrics import f1_score
preds_val = tabnet_model.predict(X_val)
f1 = f1_score(y_val, preds_val, average='weighted')  # Use 'weighted' to handle imbalanced classes
print(f"Validation F1 Score: {f1}")

Validation F1 Score: 0.3333333333333333


In [15]:
napi.download_dataset(f"{DATA_VERSION}/validation.parquet");

validation = pd.read_parquet(
    f"{DATA_VERSION}/validation.parquet",
    columns=["era", "data_type"] + feature_cols + target_cols
)
validation = validation[validation["data_type"] == "validation"]
del validation["data_type"]


2024-11-07 15:01:40,103 INFO numerapi.utils: target file already exists
2024-11-07 15:01:40,105 INFO numerapi.utils: starting download
v5.0/validation.parquet: 6.47GB [01:47, 60.4MB/s]                            


In [16]:
validation = validation[['era','target'] + top_100_features]

In [17]:
validation = validation[validation["era"].isin(validation["era"].unique()[::4])]

last_train_era = int(training_data["era"].unique()[-1])
eras_to_embargo = [str(era).zfill(4) for era in [last_train_era + i for i in range(4)]]
validation = validation[~validation["era"].isin(eras_to_embargo)]

In [18]:
validation_df = validation

In [20]:
from sklearn.metrics import classification_report

X_val = validation_df[top_100_features].values
y_val = validation_df["target"].values

bins = [0, 0.25, 0.5, 0.75, 1]
y_val_class = np.digitize(y_val, bins) - 1

preds_val_class = tabnet_model.predict(X_val)

preds_val = np.array([bins[pred] for pred in preds_val_class])

accuracy = accuracy_score(y_val_class, preds_val_class)
f1 = f1_score(y_val_class, preds_val_class, average='weighted')

print(f"Validation Accuracy: {accuracy}")
print(f"Validation F1 Score: {f1}")

# Calculate class-wise F1 score and class-wise accuracy on validation data
report = classification_report(y_val_class, preds_val_class, target_names=[str(b) for b in bins], output_dict=True)

print("\nClass-wise metrics on validation data:")
for class_label, metrics in report.items():
    if class_label in ["accuracy", "macro avg", "weighted avg"]:
        continue
    print(f"Class {class_label}:")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall: {metrics['recall']:.4f}")
    print(f"  F1 Score: {metrics['f1-score']:.4f}")
    print(f"  Support: {metrics['support']}")


Validation Accuracy: 0.49994743451553686
Validation F1 Score: 0.3332787795996623

Class-wise metrics on validation data:
Class 0:
  Precision: 0.0000
  Recall: 0.0000
  F1 Score: 0.0000
  Support: 42801.0
Class 0.25:
  Precision: 0.0000
  Recall: 0.0000
  F1 Score: 0.0000
  Support: 173656.0
Class 0.5:
  Precision: 0.4999
  Recall: 1.0000
  F1 Score: 0.6666
  Support: 432750.0
Class 0.75:
  Precision: 0.2000
  Recall: 0.0000
  F1 Score: 0.0000
  Support: 173488.0
Class 1:
  Precision: 0.0000
  Recall: 0.0000
  F1 Score: 0.0000
  Support: 42892.0


In [21]:
import cloudpickle
DATA_VERSION = "v5.0"

feature_metadata = json.load(open(f"{DATA_VERSION}/features.json"))
feature_cols = feature_metadata["feature_sets"]["medium"]

live_data = pd.read_parquet(f"{DATA_VERSION}/live.parquet", columns=top_100_features)
live_data = live_data.fillna(-1)


def predict(live_features: pd.DataFrame) -> pd.DataFrame:
    live_features = live_features[top_100_features].values

    with torch.no_grad():
        live_predictions_class = tabnet_model.predict(live_features)

    class_to_target = {0: 0, 1: 0.25, 2: 0.5, 3: 0.75, 4: 1}
    live_predictions_target = [class_to_target[pred] for pred in live_predictions_class]

    submission = pd.DataFrame({
        'id': live_data.index,
        'prediction': live_predictions_target
    })
    return submission

In [22]:
p = cloudpickle.dumps(predict)
with open("predict.pkl", "wb") as f:
    f.write(p)

In [23]:
try:
    from google.colab import files
    files.download('predict.pkl')
except:
    pass